In [1]:
import pandas as pd

df = pd.read_csv("C:/Users/KEARNEY/Desktop/새 폴더/c_project/LDH/code/CJ_master.csv")

df['review1stComment'] = df['review1stComment']+'1'
df['review2ndComment'] = df['review2ndComment']+'2'

#메뉴
start_idx = df.columns.get_loc('m_치즈감자전')
end_idx = df.columns.get_loc('m_버섯베이컨알프레도파스타')

menu= df.iloc[:,start_idx:end_idx]

# 식자재 a_혼합야채 to a_버팔로윙
ing_start_idx = df.columns.get_loc('a_혼합야채')
ing_end_idx = df.columns.get_loc('a_버팔로윙')

ingred = df.iloc[:,ing_start_idx:ing_end_idx]

# 리뷰등 공통 칼럼
# "review1stComment",, "review2ndComment" 제외함 
more_info = df[["visitorReview","blogReview","review1st", "review2nd"]]

#식당이랑 카테고리 칼럼 추가하고 순서 변경
df1 = pd.concat([menu,ingred, more_info],axis=1)

df1['category'] = df['category']
df1 = df1[['category']+ df1.columns[:-1].to_list()]

df1['식당']= df['name']
df1 = df1[['식당']+ df1.columns[:-1].to_list()]

#폐업 제거
df1 = df1.dropna()

#유사도 행렬 만들기
from sklearn.metrics.pairwise import cosine_similarity

#pos와 아닌것 분리해서 가져오기
group_non_pos = df1[df1['category'] != 'POS'].iloc[:, 2:].reset_index(drop=True)
group_pos = df1[df1['category'] == 'POS'].iloc[:, 2:].reset_index(drop=True)

#인덱스 설정
non_pos_idx= df1[df1['category'] != 'POS']['식당']
pos_idx= df1[df1['category'] == 'POS']['식당']
non_pos_idx.name= 'non pos 식당'
pos_idx.name= 'pos 식당'

#유사도 행렬 만들기
similarity_matrix = cosine_similarity(group_non_pos, group_pos)
similarity_df = pd.DataFrame(similarity_matrix, index=non_pos_idx, columns=pos_idx)

# review1stComment랑 2nd 원핫한 뒤 유사도 구한 것

In [5]:
df1['visitorReview']

0     0.061531
1    -0.554915
2    -0.577225
3    -0.283679
4    -0.479768
        ...   
64    0.021608
65   -0.574877
66   -0.650024
67   -0.075849
69   -0.304814
Name: visitorReview, Length: 67, dtype: float64

In [8]:
# "review1stComment",, "review2ndComment" 제외함 
more_info = df[["visitorReview","blogReview","review1st", "review2nd","review1stComment", "review2ndComment" ]]

#식당이랑 카테고리 칼럼 추가하고 순서 변경
df1 = pd.concat([menu,ingred, more_info],axis=1)

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

# 칼럼 a, b, c를 standardized
df1[["visitorReview","blogReview","review1st", "review2nd"]] = scaler.fit_transform(df1[["visitorReview","blogReview","review1st", "review2nd"]])


df1['category'] = df['category']
df1 = df1[['category']+ df1.columns[:-1].to_list()]

df1['식당']= df['name']
df1 = df1[['식당']+ df1.columns[:-1].to_list()]

df1= df1.dropna()

df1 = pd.get_dummies(df1, columns=['review1stComment', 'review2ndComment'])

#유사도 행렬 만들기
from sklearn.metrics.pairwise import cosine_similarity

#pos와 아닌것 분리해서 가져오기
group_non_pos = df1[(df1['category'] == 'FW') | (df1['category'] =='FO')].iloc[:, 2:].reset_index(drop=True)
group_pos = df1[(df1['category'] == 'POS') | (df1['category']=='FWPOS')].iloc[:, 2:].reset_index(drop=True)

#인덱스 설정
non_pos_idx= df1[(df1['category'] == 'FW') | (df1['category'] =='FO')]['식당']
pos_idx= df1[(df1['category'] == 'POS') | (df1['category']=='FWPOS')]['식당']
non_pos_idx.name= 'non pos 식당'
pos_idx.name= 'pos 식당'

#유사도 행렬 만들기
similarity_matrix = cosine_similarity(group_non_pos, group_pos)
similarity_df = pd.DataFrame(similarity_matrix, index=non_pos_idx, columns=pos_idx)

similarity_df.head()

pos 식당,kitchen485,감성카츠,감성타코(홍대점),광야,까사라티나,덩크트레이,돈카츠준,드렁큰비건,루나씨엘로,마루돈까스,...,진돈,치킨앤카레군,코코가츠,크레이지카츠,테이커테이블,토모토카레,토파,포트앤나달,헤키,흑심
non pos 식당,,,,,,,,,,,,,,,,,,,,,
미장플라쎄,0.568619,0.460436,0.538470,0.579595,0.125400,0.512854,0.418305,0.489421,0.563162,0.451160,...,0.446658,0.499442,0.459808,0.431056,0.586414,0.502815,0.548684,0.524235,0.286234,0.433881
이관우파스타,0.524183,0.397193,0.483494,0.588813,0.110135,0.485058,0.381449,0.483120,0.521759,0.384926,...,0.415164,0.448055,0.413596,0.390407,0.505587,0.427260,0.504891,0.501808,0.283461,0.379334
디트로이트피지스,0.604880,0.453306,0.563033,0.498365,0.100766,0.587675,0.427268,0.481039,0.547010,0.451524,...,0.455190,0.497069,0.440173,0.444710,0.517597,0.474718,0.498008,0.465332,0.292325,0.436647
피크닉마나올라나,0.464700,0.413399,0.478143,0.411931,0.110510,0.468926,0.386928,0.411024,0.429172,0.415985,...,0.405506,0.440100,0.401306,0.409162,0.490268,0.446259,0.411729,0.408965,0.296324,0.405211
키노키친,0.609719,0.498114,0.724495,0.594941,0.134109,0.540133,0.457638,0.523761,0.631570,0.486739,...,0.518305,0.553282,0.502580,0.468448,0.741913,0.535145,0.579726,0.545249,0.305572,0.438077


In [9]:
similarity_df.to_csv("similarity.csv", encoding="euc-kr")

In [30]:
#테이커 테이블, 카페루치아 정리

df = df[df['name'] !='카페루치아']

idx = df.columns.get_loc("b12_혼합야채")
df.iloc[64,idx:] = df.iloc[11,idx:]
df = pd.concat([df.iloc[:11,:],df.iloc[12:,:]],axis=0)

import numpy as np

df = df.set_index(np.array(range(len(df))))
df.loc[df['name']=='테이커테이블', 'category'] = "FWPOS"

df.to_csv('cj_master_final.csv',encoding='euc-kr',index=False)